# Case Study: Climate Data for Soweto Health Analysis

This notebook demonstrates how to extract temperature data for Soweto, South Africa for health research.

**Use Case**: Analyzing preterm births vs temperature (April 2016 - March 2022)

**Data Extracted**:
- Daily Maximum Temperature (TMax)
- Daily Mean Temperature (TMean)
- Monthly averages for correlation analysis

In [ ]:
# Import required libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine initialized successfully!")
except Exception as e:
    print(f"Error initializing Earth Engine: {e}")
    print("Please run: earthengine authenticate")

## 1. Define Study Area and Time Period

In [ ]:
# Define Soweto coordinates (approximate center)
soweto_lat = -26.2678
soweto_lon = 27.8607

# Create a point geometry for Soweto
soweto_point = ee.Geometry.Point([soweto_lon, soweto_lat])

# Create a buffer around the point (5km radius for local area)
soweto_area = soweto_point.buffer(5000)  # 5km buffer

# Define time period
start_date = '2016-04-01'
end_date = '2022-03-31'

print(f"Study Location: Soweto, South Africa ({soweto_lat}, {soweto_lon})")
print(f"Study Period: {start_date} to {end_date}")
print(f"Buffer Area: 5km radius around center point")

## 2. Extract ERA5 Temperature Data

In [ ]:
# Load ERA5 daily temperature data
era5_daily = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
    .filterDate(start_date, end_date) \
    .filterBounds(soweto_area)

# Function to convert temperature from Kelvin to Celsius
def kelvin_to_celsius(image):
    return image.select(['temperature_2m_max', 'temperature_2m_mean']) \
        .subtract(273.15) \
        .copyProperties(image, ['system:time_start'])

# Convert temperatures to Celsius
temp_data = era5_daily.map(kelvin_to_celsius)

print(f"Found {temp_data.size().getInfo()} daily temperature records")

## 3. Extract Time Series Data to DataFrame

In [ ]:
# Function to extract time series data
def extract_temperature_timeseries(collection, geometry, start_date, end_date):
    """
    Extract daily temperature time series from ERA5 data
    """
    # Create date range
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Initialize lists to store data
    date_list = []
    tmax_list = []
    tmean_list = []
    
    # Extract data in chunks to avoid memory issues
    chunk_size = 100
    total_days = len(dates)
    
    for i in range(0, total_days, chunk_size):
        end_chunk = min(i + chunk_size, total_days)
        chunk_start = dates[i].strftime('%Y-%m-%d')
        chunk_end = dates[end_chunk-1].strftime('%Y-%m-%d')
        
        # Filter collection for this chunk
        chunk_collection = collection.filterDate(chunk_start, chunk_end)
        
        # Extract values for each day in chunk
        chunk_list = chunk_collection.getRegion(geometry, 1000).getInfo()
        
        # Process chunk data
        for row in chunk_list[1:]:
            if row[4] is not None and row[5] is not None:  # Check for valid data
                date_ms = row[3]
                date_obj = datetime.fromtimestamp(date_ms / 1000)
                
                date_list.append(date_obj.date())
                tmax_list.append(row[4])  # temperature_2m_max
                tmean_list.append(row[5]) # temperature_2m_mean
        
        print(f"Processed chunk {i//chunk_size + 1}/{(total_days-1)//chunk_size + 1}")
    
    # Create DataFrame
    df = pd.DataFrame({
        'date': date_list,
        'tmax_celsius': tmax_list,
        'tmean_celsius': tmean_list
    })
    
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    return df

# Extract the time series (this may take a few minutes)
print("Extracting temperature time series...")
temp_df = extract_temperature_timeseries(temp_data, soweto_point, start_date, end_date)

print(f"\nExtracted {len(temp_df)} daily temperature records")
print("\nFirst few records:")
print(temp_df.head())

## 4. Calculate Monthly Averages

In [ ]:
# Add month and year columns
temp_df['year'] = temp_df['date'].dt.year
temp_df['month'] = temp_df['date'].dt.month
temp_df['year_month'] = temp_df['date'].dt.to_period('M')

# Calculate monthly averages
monthly_temp = temp_df.groupby('year_month').agg({
    'tmax_celsius': 'mean',
    'tmean_celsius': 'mean',
    'date': 'count'  # Count of days per month
}).round(2)

monthly_temp.columns = ['avg_tmax', 'avg_tmean', 'days_count']
monthly_temp = monthly_temp.reset_index()
monthly_temp['date'] = monthly_temp['year_month'].dt.to_timestamp()

print("Monthly temperature averages:")
print(monthly_temp.head(10))

print(f"\nTotal months: {len(monthly_temp)}")
print(f"Temperature range - Max: {temp_df['tmax_celsius'].min():.1f}°C to {temp_df['tmax_celsius'].max():.1f}°C")
print(f"Temperature range - Mean: {temp_df['tmean_celsius'].min():.1f}°C to {temp_df['tmean_celsius'].max():.1f}°C")

## 5. Data Visualization

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Soweto Temperature Analysis (April 2016 - March 2022)', fontsize=16, fontweight='bold')

# 1. Time series plot
axes[0, 0].plot(monthly_temp['date'], monthly_temp['avg_tmax'], 'r-o', label='Max Temp', markersize=3)
axes[0, 0].plot(monthly_temp['date'], monthly_temp['avg_tmean'], 'b-o', label='Mean Temp', markersize=3)
axes[0, 0].set_title('Monthly Average Temperatures')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Seasonal pattern (by month)
temp_df['month_name'] = temp_df['date'].dt.strftime('%b')
monthly_pattern = temp_df.groupby(temp_df['date'].dt.month).agg({
    'tmax_celsius': 'mean',
    'tmean_celsius': 'mean'
})

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
axes[0, 1].bar(range(1, 13), monthly_pattern['avg_tmax'], alpha=0.7, color='red', label='Max Temp')
axes[0, 1].bar(range(1, 13), monthly_pattern['avg_tmean'], alpha=0.7, color='blue', label='Mean Temp')
axes[0, 1].set_title('Seasonal Temperature Pattern')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Temperature (°C)')
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].set_xticklabels(month_names)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Temperature distribution
axes[1, 0].hist(temp_df['tmax_celsius'], bins=30, alpha=0.7, color='red', label='Max Temp')
axes[1, 0].hist(temp_df['tmean_celsius'], bins=30, alpha=0.7, color='blue', label='Mean Temp')
axes[1, 0].set_title('Temperature Distribution')
axes[1, 0].set_xlabel('Temperature (°C)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Box plot by year
yearly_data = []
years = sorted(temp_df['year'].unique())
for year in years:
    year_data = temp_df[temp_df['year'] == year]['tmean_celsius']
    yearly_data.append(year_data)

axes[1, 1].boxplot(yearly_data, labels=years)
axes[1, 1].set_title('Annual Temperature Variation')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Mean Temperature (°C)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n=== TEMPERATURE SUMMARY STATISTICS ===")
print("\nDaily Maximum Temperature (°C):")
print(temp_df['tmax_celsius'].describe().round(2))
print("\nDaily Mean Temperature (°C):")
print(temp_df['tmean_celsius'].describe().round(2))

## 6. Export Data for Health Analysis

In [ ]:
# Export daily data
daily_export = temp_df[['date', 'tmax_celsius', 'tmean_celsius']].copy()
daily_export.columns = ['date', 'daily_tmax_c', 'daily_tmean_c']

# Export monthly data
monthly_export = monthly_temp[['date', 'avg_tmax', 'avg_tmean']].copy()
monthly_export.columns = ['month', 'monthly_avg_tmax_c', 'monthly_avg_tmean_c']
monthly_export['month'] = monthly_export['month'].dt.strftime('%Y-%m')

# Save to CSV files
daily_export.to_csv('../data/soweto_daily_temperature_2016_2022.csv', index=False)
monthly_export.to_csv('../data/soweto_monthly_temperature_2016_2022.csv', index=False)

# Save to Excel with both sheets
with pd.ExcelWriter('../data/soweto_temperature_data_2016_2022.xlsx') as writer:
    daily_export.to_excel(writer, sheet_name='Daily_Data', index=False)
    monthly_export.to_excel(writer, sheet_name='Monthly_Averages', index=False)

print("Data exported successfully!")
print(f"Daily records: {len(daily_export)}")
print(f"Monthly records: {len(monthly_export)}")
print("\nFiles created:")
print("- ../data/soweto_daily_temperature_2016_2022.csv")
print("- ../data/soweto_monthly_temperature_2016_2022.csv") 
print("- ../data/soweto_temperature_data_2016_2022.xlsx")

print("\n=== SAMPLE DATA FOR HEALTH ANALYSIS ===")
print("\nMonthly averages (first 12 months):")
print(monthly_export.head(12))

## 7. Prepare for Health Outcome Analysis

This section provides template code for correlating the extracted temperature data with health outcomes.

In [ ]:
# Template for health data integration
def analyze_health_climate_correlation(health_data_file, temperature_data):
    """
    Template function for analyzing health outcomes vs temperature
    
    Parameters:
    health_data_file: CSV file with columns ['month', 'outcome_rate']
    temperature_data: DataFrame with temperature data
    """
    
    # Load health data (replace with actual data)
    # health_df = pd.read_csv(health_data_file)
    
    # For demonstration, create sample health data
    print("Creating sample health outcome data for demonstration...")
    
    # Create sample preterm birth rates (normally distributed around 10% with temperature correlation)
    sample_health = monthly_export.copy()
    
    # Simulate preterm birth rate with temperature correlation
    np.random.seed(42)
    base_rate = 0.10  # 10% baseline rate
    temp_effect = (sample_health['monthly_avg_tmean_c'] - 20) * 0.001  # Small temperature effect
    random_variation = np.random.normal(0, 0.02, len(sample_health))  # Random variation
    
    sample_health['preterm_rate'] = base_rate + temp_effect + random_variation
    sample_health['preterm_rate'] = np.clip(sample_health['preterm_rate'], 0.05, 0.20)  # Reasonable bounds
    
    # Calculate correlation
    correlation = sample_health['monthly_avg_tmean_c'].corr(sample_health['preterm_rate'])
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Time series comparison
    ax1 = axes[0]
    ax2 = ax1.twinx()
    
    dates = pd.to_datetime(sample_health['month'])
    ax1.plot(dates, sample_health['monthly_avg_tmean_c'], 'r-o', label='Temperature', markersize=4)
    ax2.plot(dates, sample_health['preterm_rate'] * 100, 'b-s', label='Preterm Rate (%)', markersize=4)
    
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Temperature (°C)', color='red')
    ax2.set_ylabel('Preterm Birth Rate (%)', color='blue')
    ax1.set_title('Temperature vs Health Outcomes Over Time')
    ax1.grid(True, alpha=0.3)
    
    # Scatter plot
    axes[1].scatter(sample_health['monthly_avg_tmean_c'], sample_health['preterm_rate'] * 100, alpha=0.7)
    axes[1].set_xlabel('Monthly Average Temperature (°C)')
    axes[1].set_ylabel('Preterm Birth Rate (%)')
    axes[1].set_title(f'Temperature vs Preterm Rate\n(Correlation: {correlation:.3f})')
    axes[1].grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(sample_health['monthly_avg_tmean_c'], sample_health['preterm_rate'] * 100, 1)
    p = np.poly1d(z)
    axes[1].plot(sample_health['monthly_avg_tmean_c'], p(sample_health['monthly_avg_tmean_c']), 'r--', alpha=0.8)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nCorrelation between temperature and preterm birth rate: {correlation:.3f}")
    
    return sample_health

# Run the analysis
health_climate_data = analyze_health_climate_correlation(None, monthly_export)

print("\n=== NEXT STEPS FOR HEALTH ANALYSIS ===")
print("1. Replace sample health data with actual health outcomes")
print("2. Perform statistical tests (chi-square, t-tests, ANOVA)")
print("3. Consider confounding factors (year, season, socioeconomic factors)")
print("4. Conduct time series analysis and seasonal decomposition")
print("5. Create publication-ready visualizations")

## 8. Data Sources and Methodology

**Data Source**: ERA5-Land Daily Aggregated Data
- **Provider**: European Centre for Medium-Range Weather Forecasts (ECMWF)
- **Spatial Resolution**: ~11 km
- **Temporal Resolution**: Daily
- **Variables**: 2m Temperature (Maximum and Mean)
- **Quality**: High-quality reanalysis data suitable for health research

**Study Area**: Soweto, South Africa
- **Coordinates**: 26.2678°S, 27.8607°E
- **Buffer**: 5km radius around center point
- **Justification**: Captures local temperature variation while maintaining spatial coherence

**Methodology**:
1. Extract daily temperature data from ERA5-Land
2. Convert from Kelvin to Celsius
3. Calculate monthly averages for health outcome correlation
4. Export data in formats suitable for statistical analysis

**Recommended Statistical Analyses**:
- Pearson/Spearman correlation for temperature-health relationships
- Time series decomposition for seasonal patterns
- Regression analysis controlling for temporal trends
- ANOVA for comparing outcomes across temperature categories